In [1]:
from common import *

In [2]:
db = load_db(download_tar("http://vision.lems.brown.edu/sites/default/files/216db.tar.gz"))

216db exists. Skipping


In [3]:
df = pd.DataFrame(data=db)
df = df.sort_values(by=["clas","num"])
print(df.clas.unique())
df

['bird' 'bone' 'brick' 'camel' 'car' 'children' 'classic' 'elephant'
 'face' 'fork' 'fountain' 'glas' 'hammer' 'heart' 'key' 'misk' 'ray'
 'turtle']


,clas,num,image,filepath
149,bird,02,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/bird02.pgm
213,bird,04,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/bird04.pgm
48,bird,07,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/bird07.pgm
203,bird,09,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/bird09.pgm
83,bird,10,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/bird10.pgm
...,...,...,...,...
193,turtle,13,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/turtle13.pgm
178,turtle,16,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/turtle16.pgm
202,turtle,17,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/turtle17.pgm
191,turtle,18,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",216db/pgms/turtle18.pgm


In [4]:
%matplotlib qt

In [5]:
chosen="turtle"

def plot_cls(chosen):

    data =df.query(f"clas=='{chosen}'")
    size = len(data)
    COLSIZE=5
    fig, axs = plt.subplots(nrows=size//COLSIZE+1, ncols=COLSIZE, figsize=(COLSIZE*6,(size//COLSIZE+1)*6))
    for ax, (i, row) in zip(axs.flatten(), data.iterrows()):
        ax.imshow(row.image)
        ax.set_title(f"{row.num}")

    fig.suptitle(chosen)
    
plot_cls(chosen)

In [6]:
for c in df.clas.unique():
    plot_cls(c)

In [ ]:
COLSIZE=12
fig, axs = plt.subplots(nrows=len(df.clas.unique()), ncols=COLSIZE, figsize=(COLSIZE*2,50))
for clas,axrow in zip(df.clas.unique(), axs):
    for ax, (i, row) in zip(axrow,df.query(f"clas=='{clas}'").iterrows()):
        ax.imshow(row.image)
        ax.set_title(f"{ row.clas }{row.num}")
        ax.axis("off")

fig.tight_layout()

In [43]:
im = df.image.iloc[0]

plt.imshow(im)
im.shape

contours, hierarchy = cv2.findContours(255-im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [44]:
def pltcnt(c, ax=None):
    if ax:
        ax.plot([p[0][0] for p in c], [ p[0][1] for p in c ])
    else:
        plt.plot([p[0][0] for p in c], [ p[0][1] for p in c ])

In [146]:
def corr(x,y):
    assert len(x) == len(y)

    out = np.zeros(len(x))
    for i in range(len(x)):
        out[i] = np.sum(np.roll(x,i)*y)
    
    return out

def sample_contour(c, n=500):
    sampling = np.linspace(0,1,num=n)
    sampled = c[np.round(sampling*(len(c)-1)).astype(int)]
    m = np.mean(sampled, axis=0)
    return np.linalg.norm(sampled-m, axis=-1).flatten()

def contour_corr(c1,c2, n=500):

    c1_norm = sample_contour(c1,n=n)
    c1_norm = c1_norm#/c1_norm.max()
    c2_norm = sample_contour(c2,n=n)
    c2_norm = c2_norm#/c2_norm.max()

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.plot(np.linspace(0, 2*np.pi, num=n),c1_norm)
    ax.plot(np.linspace(0, 2*np.pi, num=n),c2_norm)
    return corr(c1_norm, c2_norm)

In [154]:
sampling = np.linspace(0,1,num=500)
sampled = c[np.round(sampling*(len(c)-1)).astype(int)]
cc = sampled - np.mean(sampled, axis=0)

plt.figure()
pltcnt(cc)

In [156]:
plt.plot(np.mean(cc, axis=0), "rx")

In [147]:
fig, axeses = plt.subplots(2,2)

waves = []
for i, axs in enumerate(axeses):
    im = df.image.iloc[i-1]
    contours, hierarchy = cv2.findContours(255-im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    axs[0].imshow(im)
    for c in contours:
        pltcnt(c, ax=axs[0])

    c = contours[0]
    m = np.mean(c, axis=0)
    axs[0].plot(m[0,0],m[0,1], "rx")

    dists = np.linalg.norm(c-m, axis=-1).flatten()
    axeses[0][1].plot(np.hstack((dists,dists,dists)))
    axeses[0][1].axvline(len(dists), color="k")
    axeses[0][1].axvline(2*len(dists), color="k")
    waves.append(c)

axeses[1][1].plot(contour_corr(*waves))


In [49]:
np.linalg.norm(c-m, axis=-1)

array([[65.21005363],
       [64.83851412],
       [65.87230212],
       [64.82196741],
       [63.23071898],
       [63.06984732],
       [63.70533162],
       [63.59285412],
       [64.25429738],
       [62.77619848],
       [62.74179558],
       [61.32770547],
       [59.94964709],
       [57.12328139],
       [56.39315081],
       [54.97946893],
       [54.25735145],
       [52.84330356],
       [51.4261713 ],
       [50.01205495],
       [49.3185951 ],
       [47.90488877],
       [47.96415522],
       [47.30410519],
       [45.89371322],
       [46.02080759],
       [44.79007809],
       [45.00925364],
       [43.34959583],
       [43.69060857],
       [42.32521497],
       [42.72125514],
       [41.37221514],
       [41.82513603],
       [40.49563618],
       [41.00705266],
       [39.70054047],
       [38.67153292],
       [37.41713517],
       [36.65237441],
       [34.30022799],
       [34.14123346],
       [30.84294828],
       [30.79619523],
       [27.85801997],
       [27

In [ ]:
def show_pca(image, ax=plt):
    M = cv2.moments(255*(image==0).astype(np.uint8))

    p = (
        int(M["m10"] / M["m00"]),
        int(M["m01"] / M["m00"])
    )


    ax.imshow(image)
    ax.plot(*p, "rx")
    ax.plot(*np.mean(np.where(image==0), axis=-1)[::-1], "gx")

    # contours, hierarchy = cv2.findContours(255-image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # c = contours[0]
    # A = c[:,0,:]
    # center = np.mean(A,axis=0)
    # A = A - center
    # u, s, vt = np.linalg.svd(A)

    A = np.vstack(np.where(image==0)[::-1])
    center = np.mean(A,axis=-1)
    A = A.T - center
    u, s, vt = np.linalg.svd(A)

    if np.linalg.det(vt) < 0:
        vt[0,:] = vt[0,:]*-1

    ax.arrow(*center, *20*vt[1,:], width=1)
    ax.arrow(*center, *20*vt[0,:], width=1)
    return vt, center

for im in db.query("clas=='glas'").image:
    fig, ax = plt.subplots(1,2, figsize=(15,9))
    vt, center = show_pca(im, ax=ax[0])

    r= vt
    c = center
    dsize = (250,250)

    center_d = (dsize[0]/2, dsize[1]/2)
    W = np.zeros((2,3))
    W[:2,:2] = r 
    W[:2,2] = -r@c  + center_d

    dsize = (250,250)
    im2 = cv2.warpAffine(255-im, W, dsize=dsize)
    show_pca(255-im2, ax[1])
    # vt = show_pca(im2, ax=ax[1])

